In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
start="2022-01-01"
end="2026-01-01"
market_ticker = '^AXJO'
tickers = ['CBA', 'NAB', 'WBC', 'ANZ']
tickers = [market_ticker] + [i + '.AX' for i in tickers]


In [ ]:
def download_data(stocks, start, end):
  df = yf.download(stocks, start=start, end=end, auto_adjust=False)[["Close"]]
  market_col = ("Close", market_ticker)
  market = df[market_col]
  df = df.drop(market_col, axis=1)
  return (df, market)

In [ ]:
df, market = download_data(tickers, start, end)

In [ ]:
def get_beta(returns, market_returns):
    betas = {}

    for col in returns.columns:
        aligned = returns[col].align(market_returns, join="inner")
        s, m = aligned

        cov = np.cov(s, m)
        betas[col] = cov[0, 1] / cov[1, 1]

    return pd.Series(betas, name="Beta")

def matrix_beta(returns, market_returns):
    returns, market_returns = returns.align(market_returns, join="inner", axis=0)

    Y = returns.to_numpy()                  
    x = market_returns.to_numpy()[:, None]  

    X = np.column_stack([np.ones(len(x)), x])
    beta = np.linalg.lstsq(X, Y, rcond=None)[0]
    betas = pd.Series(beta[1], index=returns.columns, name="Beta")
    return betas


In [ ]:
log_ret = np.log(df["Close"]/df["Close"].shift()).dropna()
market_log_ret = np.log(market/market.shift()).dropna()
#betas = get_beta(log_ret, market_log_ret)
betas = matrix_beta(log_ret, market_log_ret)

In [ ]:
units = np.array([10]*(len(tickers)-1))
prices = df["Close"].iloc[-1].to_numpy()
market_price = market.iloc[-1]
value = units*prices
w = [round(val/sum(value), 2) for val in value]


In [ ]:
Portfolio = pd.DataFrame(
    {
        "tickers": tickers[:-1],
        "Type": "S",
        "direction": "long",
        "stock price": prices,
        "price": prices,
        "units": units,
        "value": value,
        "beta": betas,
        "weighted beta": betas*w
    }
)


In [ ]:
Portfolio["delta"] = Portfolio["units"]

In [ ]:
Options = [{'option':'CBA0Z8', 'underlying':'CBA', 'price':3.950, 'units': 2, 'delta': 0.627, 'direction': 'Short', 'type': 'Call'}]

In [ ]:
for row in Options:
    Portfolio.loc[row['option']] = {
        "Type": "O",
        "direction": row["direction"],
        "stock price": Portfolio.loc[row["underlying"], "price"],
        "price": row["price"],
        "units": row["units"] * (1 if row["direction"] == "Long" else -1),
        "value": row["price"] * row["units"] * 100,
        "beta": betas[row["underlying"]],
        "weight": np.nan,
        "weighted beta": (
            row["delta"] * row["units"] * 100
            if row["direction"] == "Long"
            else -row["delta"] * row["units"] * 100
        )
    }


In [ ]:
Portfolio["weighted delta (point)"] = round(Portfolio['beta'] * (Portfolio['stock price']/market_price) * Portfolio['delta'],2)
Portfolio["weighted delta (1%)"] = round(Portfolio['beta'] * (Portfolio['stock price']) * Portfolio['delta'] * 0.01,2)
Portfolio.loc['total', ['value', 'weighted delta (point)', 'weighted delta (1%)']] \
= Portfolio[['value','weighted delta (point)', 'weighted delta (1%)']].sum()
Portfolio